In [2]:
import pandas as pd

df = pd.read_csv('sales_2024-09-01.csv')
df.head()



,store_id,store_name,product_category,date,unit_sales,dollar_sales,store_zip,promotion_flag
0,LA002,Store_2,Household,2024-09-01,16.0,96.28,90004,True
1,LA001,Store_1,Beverages,2024-09-01,14.0,86.63,90004,True
2,LA008,Store_8,Produce,2024-09-01,4.0,79.99,90001,False
3,LA010,Store_10,Beverages,2024-09-01,22.0,140.45,90005,True
4,LA006,Store_6,Snacks,2024-09-01,17.0,50.03,90003,False


In [ ]:
top_3_products = (
    df.groupby("store_name", as_index=False)["dollar_sales"]
      .sum()
      .sort_values(by="dollar_sales", ascending=False)
      .head(3)
)

print(top_3_products)

  store_name  dollar_sales
5    Store_5       1682.61
9    Store_9       1539.90
0    Store_1       1211.51


In [5]:
daily_store_rev = (
    df.groupby(["date", "store_id", "store_name"], as_index=False)["dollar_sales"]
      .sum()
      .rename(columns={"dollar_sales": "total_revenue"})
)

top3 = (

    daily_store_rev.sort_values("total_revenue", ascending=False)
    .head(3)
    .reset_index(drop=True)
)

top3


,date,store_id,store_name,total_revenue
0,2024-09-01,LA005,Store_5,1682.61
1,2024-09-01,LA009,Store_9,1539.90
2,2024-09-01,LA001,Store_1,1211.51


In [6]:
import glob 
folder = '/Users/as-mac-1346/Desktop/Classes/DataWarehouse/DAY-04/LA_Retail_Sales_By_Day'
file_paths = glob.glob(f"{folder}/*.csv")
print("Files found:", len(file_paths))
file_paths[:5]

Files found: 30


['/Users/as-mac-1346/Desktop/Classes/DataWarehouse/DAY-04/LA_Retail_Sales_By_Day/sales_2024-09-27.csv',
 '/Users/as-mac-1346/Desktop/Classes/DataWarehouse/DAY-04/LA_Retail_Sales_By_Day/sales_2024-09-26.csv',
 '/Users/as-mac-1346/Desktop/Classes/DataWarehouse/DAY-04/LA_Retail_Sales_By_Day/sales_2024-09-18.csv',
 '/Users/as-mac-1346/Desktop/Classes/DataWarehouse/DAY-04/LA_Retail_Sales_By_Day/sales_2024-09-30.csv',
 '/Users/as-mac-1346/Desktop/Classes/DataWarehouse/DAY-04/LA_Retail_Sales_By_Day/sales_2024-09-24.csv']

In [7]:
df = pd.concat((pd.read_csv(fp) for fp in file_paths),ignore_index = True)
df.head()

,store_id,store_name,product_category,date,unit_sales,dollar_sales,store_zip,promotion_flag
0,LA009,Store_9,Household,2024-09-27,5.0,45.74,90003,True
1,LA002,Store_2,Household,2024-09-27,10.0,44.77,900XX,False
2,LA003,Store_3,Household,2024-09-27,40.0,504.92,90003,True
3,LA007,Store_7,Personal Care,2024-09-27,32.0,80.98,90003,False
4,LA003,Store_3,Produce,2024-09-27,17.0,131.84,90003,True


In [8]:
top_3_products = (
    df.groupby("store_name", as_index=False)["dollar_sales"]
      .sum()
      .sort_values(by="dollar_sales", ascending=False)
      .head(3)
)

print(top_3_products)

  store_name  dollar_sales
1   Store_10      26671.40
6    Store_6      23636.70
9    Store_9      22413.21


In [12]:
import requests
from datetime import datetime

In [13]:
LAT, LON = 12.9716, 77.5946  # Coordinates for Bengaluru
LAT, LON

(12.9716, 77.5946)

In [14]:
unix_timestamp = int(datetime.now().timestamp())
unix_timestamp

1770195242

In [56]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Fetch the API key
API_KEY = os.getenv("API_KEY")

In [57]:
url = (
    "https://api.openweathermap.org/data/3.0/onecall/timemachine"
    f"?lat={LAT}&lon={LON}&dt={unix_timestamp}&appid={API_KEY}&units=imperial"
)
url

'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=12.9716&lon=77.5946&dt=1770195242&appid=5883b9ac08253fbd080c073b1832a6a8&units=imperial'

In [58]:
response = requests.get(url)
response.status_code

200

In [25]:
weather_data = []

if response.status_code == 200:
    data = response.json()
    today = datetime.now().date()

    day_weather = {
        "date": today,
        "temp": data["data"][0]["temp"],
        "humidity": data["data"][0]["humidity"],
        "weather_main": data["data"][0]["weather"][0]["main"],
        "weather_desc": data["data"][0]["weather"][0]["description"],
    }
    weather_data.append(day_weather)
else:
    print("Failed to fetch data for today")

weather_data

[{'date': datetime.date(2026, 2, 4),
  'temp': 81.34,
  'humidity': 31,
  'weather_main': 'Clear',
  'weather_desc': 'clear sky'}]

In [26]:

import getpass

# Requires: pip install mysql-connector-python
import mysql.connector


In [46]:
host = "localhost"
port = 3306
database = "retail_db"

user = input("MySQL username: ")
password = getpass.getpass("MySQL password: ")

conn = mysql.connector.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

print("Connected successfully.")

Connected successfully.


In [50]:
def run_query(sql: str, params=None, max_rows: int = 20):
    cur = conn.cursor()
    cur.execute(sql, params or ())
    rows = cur.fetchmany(max_rows)
    cols = [d[0] for d in cur.description] if cur.description else []
    cur.close()
    return cols, rows

In [51]:
cols, rows = run_query("SELECT * FROM orders LIMIT 10;")

print(cols)
for r in rows:
    print(r)

['order_id', 'order_date', 'order_customer_id', 'order_status']
(1, datetime.datetime(2013, 7, 25, 0, 0), 11599, 'CLOSED')
(2, datetime.datetime(2013, 7, 25, 0, 0), 256, 'PENDING_PAYMENT')
(3, datetime.datetime(2013, 7, 25, 0, 0), 12111, 'COMPLETE')
(4, datetime.datetime(2013, 7, 25, 0, 0), 8827, 'CLOSED')
(5, datetime.datetime(2013, 7, 25, 0, 0), 11318, 'COMPLETE')
(6, datetime.datetime(2013, 7, 25, 0, 0), 7130, 'COMPLETE')
(7, datetime.datetime(2013, 7, 25, 0, 0), 4530, 'COMPLETE')
(8, datetime.datetime(2013, 7, 25, 0, 0), 2911, 'PROCESSING')
(9, datetime.datetime(2013, 7, 25, 0, 0), 5657, 'PENDING_PAYMENT')
(10, datetime.datetime(2013, 7, 25, 0, 0), 5648, 'PENDING_PAYMENT')


In [40]:
cols, rows = run_query("SELECT COUNT(*) AS total_orders FROM orders;")
print(cols)
print(rows[0])

['total_orders']
(68883,)


In [41]:
cols, rows = run_query("DESC orders;", max_rows=200)

print(cols)
for r in rows:
    print(r)

['Field', 'Type', 'Null', 'Key', 'Default', 'Extra']
('order_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('order_date', 'datetime', 'NO', '', None, '')
('order_customer_id', 'int', 'NO', '', None, '')
('order_status', 'varchar(45)', 'NO', 'MUL', None, '')


In [52]:

import pandas as pd

sales_df = pd.read_json("LA_Retail_Sales.json", lines=True)
sales_df.head()

,store_id,store_name,product_category,date,unit_sales,dollar_sales,store_zip,promotion_flag
0,LA002,Store_2,Household,2024-09-01,16.0,96.28,90004,True
1,LA001,Store_1,Beverages,2024-09-01,14.0,86.63,90004,True
2,LA004,Store_4,Produce,2024-09-15,18.0,298.26,90003,True
3,LA003,Store_3,Household,2024-09-07,7.0,25.69,90001,True
4,LA008,Store_8,Beverages,2024-09-18,25.0,85.46,90005,False


In [53]:
sales_df["month"] = sales_df["date"].astype(str).str[:7]

monthly_store_rev = (
    sales_df.groupby(["month", "store_id", "store_name"], as_index=False)["dollar_sales"]
            .sum()
            .rename(columns={"dollar_sales": "total_revenue"})
)

top3_per_month = (
    monthly_store_rev.sort_values(["month", "total_revenue"], ascending=[True, False])
                     .groupby("month", as_index=False)
                     .head(3)
                     .reset_index(drop=True)
)

top3_per_month

,month,store_id,store_name,total_revenue
0,2024-09,LA010,Store_10,26671.40
1,2024-09,LA006,Store_6,23636.70
2,2024-09,LA009,Store_9,22413.21


In [54]:
import pandas as pd

sales_df = pd.read_parquet("LA_Retail_Sales.parquet", engine="fastparquet")
sales_df.head()

,store_id,store_name,product_category,date,unit_sales,dollar_sales,store_zip,promotion_flag
0,LA002,Store_2,Household,2024-09-01,16.0,96.28,90004,True
1,LA001,Store_1,Beverages,2024-09-01,14.0,86.63,90004,True
2,LA004,Store_4,Produce,2024-09-15,18.0,298.26,90003,True
3,LA003,Store_3,Household,2024-09-07,7.0,25.69,90001,True
4,LA008,Store_8,Beverages,2024-09-18,25.0,85.46,90005,False


In [55]:
sales_df["month"] = sales_df["date"].astype(str).str[:7]

monthly_store_rev = (
    sales_df.groupby(["month", "store_id", "store_name"], as_index=False)["dollar_sales"]
            .sum()
            .rename(columns={"dollar_sales": "total_revenue"})
)

top3_per_month = (
    monthly_store_rev.sort_values(["month", "total_revenue"], ascending=[True, False])
                     .groupby("month", as_index=False)
                     .head(3)
                     .reset_index(drop=True)
)

top3_per_month

,month,store_id,store_name,total_revenue
0,2024-09,LA010,Store_10,26671.40
1,2024-09,LA006,Store_6,23636.70
2,2024-09,LA009,Store_9,22413.21


In [68]:
customer_reviews = pd.read_parquet("customer_reviews_with_sentiment.parquet", engine="fastparquet")
customer_reviews.head(10)



,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,16199106,R203HPW78Z7N4K,B0067WNSZY,633038551,"FGGF3032MW Gallery Series 30"" Wide Freestandin...",Major Appliances,5,0,0,N,Y,"If you need a new stove, this is a winner.",What a great stove. What a wonderful replacem...,2015-08-31,POSITIVE
0,US,16374060,R2EAIGVLEALSP3,B002QSXK60,811766671,Best Hand Clothes Wringer,Major Appliances,5,1,1,N,Y,Five Stars,worked great,2015-08-31,POSITIVE
0,US,15322085,R1K1CD73HHLILA,B00EC452R6,345562728,Supco SET184 Thermal Cutoff Kit,Major Appliances,5,0,0,N,Y,Fast Shipping,Part exactly what I needed. Saved by purchasi...,2015-08-31,POSITIVE
0,US,32004835,R2KZBMOFRMYOPO,B00MVVIF2G,563052763,Midea WHS-160RB1 Compact Single Reversible Doo...,Major Appliances,5,1,1,N,Y,Five Stars,Love my refrigerator! ! Keeps everything cold...,2015-08-31,POSITIVE
0,US,25414497,R6BIZOZY6UD01,B00IY7BNUW,874236579,Avalon Bay Portable Ice Maker,Major Appliances,5,0,0,N,Y,Five Stars,No more running to the store for ice! Works p...,2015-08-31,POSITIVE
0,US,36311751,R1MCXZFNF8E7Y0,B0033X29CI,294467812,Danby Freestanding Wine Cooler,Major Appliances,1,0,0,N,Y,Piece of Junk,It would not cool below 55 degrees and has now...,2015-08-31,NEGATIVE
0,US,30920961,R3EMB3E3ODR6BW,B005R597HA,183784715,Avanti 110-Volt Automatic Portable Compact Dry...,Major Appliances,5,2,2,N,Y,Works awesome for apt size 110 dryer,Works awesome for apt size 110 dryer. Handles ...,2015-08-31,POSITIVE
0,US,52491265,RJTONVTTOPJ5S,B00MO6V8Y0,960251524,Danby products,Major Appliances,5,0,0,N,Y,Five Stars,exactly what I wanted!,2015-08-31,POSITIVE
0,US,48166169,R21U5QZ2CQECUM,B00HT39QDI,992475314,3 Pack Tier1 MSWF GE SmartWater Replacement Re...,Major Appliances,4,0,0,N,Y,Four Stars,AS advertised,2015-08-31,POSITIVE
0,US,50394924,RL2BBC51H89DH,B00LESFZ52,1641606,"True TSSU-60-16 60"" 2 Door Refrigerated Sandwi...",Major Appliances,4,0,0,N,Y,but has poor insulation in the top,"It works as advertised, but has poor insulatio...",2015-08-31,MIXED


In [72]:
top10 = customer_reviews.sort_values(by="star_rating",ascending = False).head(10)
top10

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,16199106,R203HPW78Z7N4K,B0067WNSZY,633038551,"FGGF3032MW Gallery Series 30"" Wide Freestandin...",Major Appliances,5,0,0,N,Y,"If you need a new stove, this is a winner.",What a great stove. What a wonderful replacem...,2015-08-31,POSITIVE
0,US,20588146,R2BI92924PW5TM,B006JJKACY,62041407,Vinotemp 5.12 cu. ft Outdoor Refrigerator,Major Appliances,5,0,0,N,Y,great frige,Bought this last fall for our new outdoor kitc...,2014-08-20,POSITIVE
0,US,51566862,RQTOU624TR4RN,B005C48JX4,292336325,GE Refrigerator Temperature Sensor - GE WR55X1...,Major Appliances,5,0,0,N,N,Five Stars,easy to install,2014-08-21,POSITIVE
0,US,9928945,R3NTS599UQ4U9U,B0026SCUS4,981155460,Whirlpool Part Number 9708180: Planetary (Empi...,Major Appliances,5,0,0,N,N,Part Came quickly. Fit perfectly. Works well.,Part Came quickly. Fit perfectly. Works well.,2014-08-21,POSITIVE
0,US,47820311,R1L7O42HA9AH9Q,B005BWZNUY,464748719,Ramblewood GC2-43P (LPG/Propane Gas) high effi...,Major Appliances,5,1,1,N,N,Five Stars,Arrived just as expected. Works great,2014-08-21,POSITIVE
0,US,29798726,R3MHWI9ED9GWGM,B007HDDMNK,699721961,Danby DDW1899WP 8 Place Setting Portable Dishw...,Major Appliances,5,0,0,N,Y,Pretty good but not pretty,Arrived quicker than I could have ordered loca...,2012-12-08,POSITIVE
0,US,37551013,R6SPSMFP5SX4O,B003SQ7I5S,835944606,MobileWasher - Hand Operating Washing Machine ...,Major Appliances,5,1,1,N,Y,Five Stars,works great,2014-08-21,POSITIVE
0,US,46622260,R3NLYV54MHEUAM,B0039V7JFG,184993957,LG 3.6 CF FRONT LOAD WASHER DRYER COMBO,Major Appliances,5,5,5,N,Y,Replace Mom with a magical laundry box!,This is THE amazing magical laundry machine! L...,2014-08-21,POSITIVE
0,US,45663593,R1JYF0YXRDZI22,B001DHKBXU,759893791,Whirlpool Washer Lid Switch 3949238,Major Appliances,5,0,0,N,Y,Five Stars,"This thing works beautifully, love it! The cos...",2014-08-20,POSITIVE
0,US,47559563,R1I0Q4PMDF5B46,B003SQ7I5S,835944606,MobileWasher - Hand Operating Washing Machine ...,Major Appliances,5,1,1,N,Y,Best machine for washing cloths and workout to...,Best machine for washing cloths and workout to...,2014-08-20,POSITIVE


In [70]:
reviews_df = pd.read_parquet("customer_reviews_with_sentiment.parquet", engine="fastparquet")
reviews_df.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,16199106,R203HPW78Z7N4K,B0067WNSZY,633038551,"FGGF3032MW Gallery Series 30"" Wide Freestandin...",Major Appliances,5,0,0,N,Y,"If you need a new stove, this is a winner.",What a great stove. What a wonderful replacem...,2015-08-31,POSITIVE
41150,US,16374060,R2EAIGVLEALSP3,B002QSXK60,811766671,Best Hand Clothes Wringer,Major Appliances,5,1,1,N,Y,Five Stars,worked great,2015-08-31,POSITIVE
41134,US,15322085,R1K1CD73HHLILA,B00EC452R6,345562728,Supco SET184 Thermal Cutoff Kit,Major Appliances,5,0,0,N,Y,Fast Shipping,Part exactly what I needed. Saved by purchasi...,2015-08-31,POSITIVE
41135,US,32004835,R2KZBMOFRMYOPO,B00MVVIF2G,563052763,Midea WHS-160RB1 Compact Single Reversible Doo...,Major Appliances,5,1,1,N,Y,Five Stars,Love my refrigerator! ! Keeps everything cold...,2015-08-31,POSITIVE
41137,US,25414497,R6BIZOZY6UD01,B00IY7BNUW,874236579,Avalon Bay Portable Ice Maker,Major Appliances,5,0,0,N,Y,Five Stars,No more running to the store for ice! Works p...,2015-08-31,POSITIVE


In [71]:
top10_by_star = (
    reviews_df[["product_title", "star_rating", "sentiment", "review_headline", "review_body"]]
      .sort_values("star_rating", ascending=False)
      .head(10)
      .reset_index(drop=True)
)

top10_by_star


,product_title,star_rating,sentiment,review_headline,review_body
0,"FGGF3032MW Gallery Series 30"" Wide Freestandin...",5,POSITIVE,"If you need a new stove, this is a winner.",What a great stove. What a wonderful replacem...
1,Vinotemp 5.12 cu. ft Outdoor Refrigerator,5,POSITIVE,great frige,Bought this last fall for our new outdoor kitc...
2,GE Refrigerator Temperature Sensor - GE WR55X1...,5,POSITIVE,Five Stars,easy to install
3,Whirlpool Part Number 9708180: Planetary (Empi...,5,POSITIVE,Part Came quickly. Fit perfectly. Works well.,Part Came quickly. Fit perfectly. Works well.
4,Ramblewood GC2-43P (LPG/Propane Gas) high effi...,5,POSITIVE,Five Stars,Arrived just as expected. Works great
5,Danby DDW1899WP 8 Place Setting Portable Dishw...,5,POSITIVE,Pretty good but not pretty,Arrived quicker than I could have ordered loca...
6,MobileWasher - Hand Operating Washing Machine ...,5,POSITIVE,Five Stars,works great
7,LG 3.6 CF FRONT LOAD WASHER DRYER COMBO,5,POSITIVE,Replace Mom with a magical laundry box!,This is THE amazing magical laundry machine! L...
8,Whirlpool Washer Lid Switch 3949238,5,POSITIVE,Five Stars,"This thing works beautifully, love it! The cos..."
9,MobileWasher - Hand Operating Washing Machine ...,5,POSITIVE,Best machine for washing cloths and workout to...,Best machine for washing cloths and workout to...
